## Tarea 1: Introducción a Spark

Fragoso Alvarado Daniel | 316049054

In [4]:
import pandas as pd
import pyspark
# Cargar Funciones Extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Tarea_1').getOrCreate()

In [16]:
# Cargar Los Datos:

fifa18 = spark.sparkContext.textFile('players_18.csv')

In [17]:
fifa18

players_18.csv MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [18]:
# Vamos a Observar la primera fila (titulos) de los datos:

for renglon in fifa18.take(1):
    print(renglon)

sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defe

In [21]:
# Vamos a eliminar la primera fila, que es el nombre de las columnas
# para así solo obtener los datos de los jugadores.

columnas = fifa18.take(1)[0]
# Reasignar los datos

fifa18 = fifa18.filter(lambda renglon: renglon != columnas)

Vamos a gnerar una función que nos ayude a agregar cada registro en tuplas que tegan los campos de nuestro ínteres. En este caso serán: `nombre`, `edad`, `estatura`, `peso`, `nacionalidad`, `equipo`, `rank`.

In [23]:
# Vamos a definir una función que agregue los datos deseados en una tupla

def separar_campos(renglon):
    # Vamos a separar la tupla con comas
    separacion = renglon.split(',')
    # Extracción de los campos
    nombre = str(separacion[2])
    edad = int(separacion[4])
    estatura = int(separacion[6])
    peso = int(separacion[7])
    nacionalidad = str(separacion[8])
    equipo = str(separacion[9])
    rank = int(separacion[10])
    # Generamos la Tupla
    return (nombre, edad, estatura, peso, nacionalidad, equipo, rank)
    

In [25]:
# Aplicamos la funcion anuestrso datos

fifa18_lim = fifa18.map(separar_campos)

# Observemos como se observan:

for renglon in fifa18_lim.take(1):
    print(renglon)

('Cristiano Ronaldo', 32, 185, 80, 'Portugal', 'Real Madrid', 94)


### Operaciones - RDD's

Vamos a realizar los ejemplos de vistos en clase, y posteriormente vamos a realizar dos operaciones nuevas:



*Ejemplo 1*: ¿Cuantos Jugadores existen por País?

In [27]:
# Vamos a seleccionar únicamete los campos de nacionalidad
nacionalidades = fifa18_lim.map(lambda renglon: (renglon[4], 1))
# Vamos a sumar todos los valores a partir del nombre del País
nacionalidades = nacionalidades.reduceByKey(lambda n, m: n+m)

# Vamos a visualizar los datos despues de aplicar la acción collect() para obtener 
# todo el conntenido de las nacionalidades que se calculó antes

nacionalidades_res = nacionalidades.collect()

# Visualización
for renglon in nacionalidades_res[:20]:
    print(renglon)

('Portugal', 350)
('Argentina', 964)
('Brazil', 799)
('Uruguay', 147)
('Poland', 320)
('Belgium', 274)
('Slovenia', 64)
('France', 958)
('Netherlands', 422)
('Gabon', 12)
('Denmark', 342)
('Slovakia', 64)
('Austria', 267)
('Greece', 97)
('Bosnia Herzegovina', 56)
('Costa Rica', 28)
('Switzerland', 232)
('Algeria', 56)
('Ecuador', 37)
('Montenegro', 25)


*Ejemplo 2*: Ordenar de forma ascendente y desendente

In [29]:
# Ordenaremos mediante SortBy() de forma desendente
nacionalidades_des = nacionalidades.sortBy(lambda renglon: renglon[1], ascending = True)

for renglon in nacionalidades_des.take(5):
    print(renglon)

('Libya', 1)
('Guatemala', 1)
('Eritrea', 1)
('Somalia', 1)
('Mauritius', 1)


In [30]:
# Ordenaremos mediante SortBy() de forma ascendente
nacionalidades_des = nacionalidades.sortBy(lambda renglon: renglon[1], ascending = False)

for renglon in nacionalidades_des.take(5):
    print(renglon)

('England', 1608)
('Germany', 1141)
('Spain', 1002)
('Argentina', 964)
('France', 958)


In [31]:
nacionalidades.count()

164

**Ejemplo 3**: ¿Cuantos equipos se encuentran en la base de datos?

In [37]:
# Vamos a seleccionar únicamente los campos de equipo (5)
equipos = fifa18_lim.map(lambda renglon: (renglon[5], 1))
# Vamos a sumar todos los valores a partir de el nombre del País
equipos = equipos.reduceByKey(lambda n, m: n+m)

# Visualización
for renglon in equipos.take(10):
    print(renglon)

('FC Bayern München', 26)
('Manchester United', 33)
('Chelsea', 33)
('Manchester City', 33)
('Atlético Madrid', 32)
('Milan', 31)
('Borussia Dortmund', 33)
('Bayer 04 Leverkusen', 26)
('Athletic Club de Bilbao', 28)
('Torino', 27)


In [35]:
# Usemos la accion count() para obtener el número de equipos:

equipos.count()

677

**Ejemplo 4**: De los 677 equipos, ¿cuales son el top 10?, basandonos en el rank de sus jugadores:

In [42]:
# Para esta consulta vamos a seleccionar el equipo (5), y el rank(6)

equipos_best = fifa18_lim.map(lambda renglon: (renglon[5], renglon[6]))
# Vamos a sumar todos los valores a apartir del nombre del País
equipos_best = equipos_best.reduceByKey(lambda n, m: n+m)

# Visualización
for renglon in equipos_best.take(5):
    print(renglon)

('FC Bayern München', 2065)
('Manchester United', 2563)
('Chelsea', 2525)
('Manchester City', 2446)
('Atlético Madrid', 2428)


In [96]:
# Notemos que no todos los equipos tienen jugadores dentro de la base de datos en la misma 
# medida, por lo que vamos a filtrar para quedarnos con aquellos equipos de más de 30 juguares
# con el fin de que no haya un sesgo que beneficie a ciertos equipos

equipos_imp = equipos.filter(lambda renglon: renglon[1] > 30)

# Seleccionaremos los equipos en una nueva variable, para lo que usaremos map()

equipos_imp_nombres = equipos_imp.map(lambda x: x[0])

In [97]:
for x in equipos_imp_nombres.take(5):
    print(x)

Manchester United
Chelsea
Manchester City
Atlético Madrid
Milan


In [98]:
# Convertimos a lista para poder iterar
equipos_imp_nombres = equipos_imp_nombres.take(equipos_imp_nombres.count())

In [104]:
# Ahora vamos a fltrar los datos del rank de los equipos:
equipos_best = equipos_best.filter(lambda renglon: renglon[0] in equipos_imp_nombres)

In [113]:
# Ahora tenemos que integrar el número de jugadores de cada equipo:
# Recordemos que ese conjunto de datos ya lo tenemos en 'equipos_imp'

# Usaremos la función join():
equipos_total = equipos_best.join(equipos_imp)

equipos_total.take(5)

[('Manchester United', (2563, 33)),
 ('Chelsea', (2525, 33)),
 ('Manchester City', (2446, 33)),
 ('Atlético Madrid', (2428, 32)),
 ('Milan', (2343, 31))]

In [140]:
# Ahora tenemos que integrar el número de jugadores de cada equipo:
# Recordemos que ese conjunto de datos ya lo tenemos en 'equipos_imp'

# Vamos a definir una función que agregue los datos deseados en una tupla

def promedio(renglon):
    # Extracción de los campos
    equipo = renglon[0]
    rank = int(renglon[1][0])
    jugadores = int(renglon[1][1])
    # Generamos la Tupla
    return (equipo, rank/jugadores)


In [141]:
# Aplicamos la funcion a nuestros datos
ranking = equipos_total.map(promedio)

# Observemos 
ranking.take(5)

[('Manchester United', 77.66666666666667),
 ('Chelsea', 76.51515151515152),
 ('Manchester City', 74.12121212121212),
 ('Atlético Madrid', 75.875),
 ('Milan', 75.58064516129032)]

In [142]:
# Ordenaremos mediante SortBy() de forma ascendente
mejores_equipos = ranking.sortBy(lambda renglon: renglon[1], ascending = False)

for renglon in mejores_equipos.take(10):
    print(renglon)

('Manchester United', 77.66666666666667)
('Chelsea', 76.51515151515152)
('Liverpool', 75.9090909090909)
('Atlético Madrid', 75.875)
('Arsenal', 75.75757575757575)
('Borussia Dortmund', 75.63636363636364)
('Milan', 75.58064516129032)
('Everton', 74.72727272727273)
('Watford', 74.18181818181819)
('Manchester City', 74.12121212121212)


### Operaciones - DataFrame

Primero haremos la importación de los datos:

In [143]:
df_fifa18 = spark.read.csv('players_18.csv', header = True, inferSchema= True)

# Seleccionemos el conjunto de columnas que nos interesan
df_fifa18 = df_fifa18.select('short_name','age', 'height_cm', 'weight_kg', 'nationality', 'club', 'value_eur', 'preferred_foot')

# Visualizamos
df_fifa18.show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|Cristiano Ronaldo| 32|      185|       80|   Portugal|        Real Madrid| 95500000|         Right|
|         L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|105000000|          Left|
|           Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|123000000|         Right|
|         M. Neuer| 31|      193|       92|    Germany|  FC Bayern München| 61000000|         Right|
|        L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona| 97000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



In [144]:
# Y podemos transformarlo a un DataFrame de Pandas
df_peq = df_fifa18.limit(10)
df_pandas = df_peq.toPandas()
df_pandas

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,Cristiano Ronaldo,32,185,80,Portugal,Real Madrid,95500000,Right
1,L. Messi,30,170,72,Argentina,FC Barcelona,105000000,Left
2,Neymar,25,175,68,Brazil,Paris Saint-Germain,123000000,Right
3,M. Neuer,31,193,92,Germany,FC Bayern München,61000000,Right
4,L. Suárez,30,182,86,Uruguay,FC Barcelona,97000000,Right
5,R. Lewandowski,28,185,79,Poland,FC Bayern München,92000000,Right
6,De Gea,26,193,76,Spain,Manchester United,64500000,Right
7,E. Hazard,26,173,76,Belgium,Chelsea,90500000,Right
8,Sergio Ramos,31,183,75,Spain,Real Madrid,52000000,Right
9,G. Higuaín,29,184,87,Argentina,Juventus,77000000,Right


In [145]:
# O bien podemos resumirlo:

df_fifa18.limit(10).toPandas()

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,Cristiano Ronaldo,32,185,80,Portugal,Real Madrid,95500000,Right
1,L. Messi,30,170,72,Argentina,FC Barcelona,105000000,Left
2,Neymar,25,175,68,Brazil,Paris Saint-Germain,123000000,Right
3,M. Neuer,31,193,92,Germany,FC Bayern München,61000000,Right
4,L. Suárez,30,182,86,Uruguay,FC Barcelona,97000000,Right
5,R. Lewandowski,28,185,79,Poland,FC Bayern München,92000000,Right
6,De Gea,26,193,76,Spain,Manchester United,64500000,Right
7,E. Hazard,26,173,76,Belgium,Chelsea,90500000,Right
8,Sergio Ramos,31,183,75,Spain,Real Madrid,52000000,Right
9,G. Higuaín,29,184,87,Argentina,Juventus,77000000,Right


In [146]:
#Para poder revisar la organización de nuestro DataFrame y el data type de cada campo podemos usarlo siguiente

df_fifa18.printSchema()

root
 |-- short_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- nationality: string (nullable = true)
 |-- club: string (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- preferred_foot: string (nullable = true)



Podemos renombrar Columnas:

In [148]:
#Para tener resultados permanentes, debemos guardar la salida en otro dataframe.
df_colum = df_fifa18.withColumnRenamed('value_eur','valor_en_euros')
df_colum.show(5)

+-----------------+---+---------+---------+-----------+-------------------+--------------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|valor_en_euros|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+--------------+--------------+
|Cristiano Ronaldo| 32|      185|       80|   Portugal|        Real Madrid|      95500000|         Right|
|         L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|     105000000|          Left|
|           Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|     123000000|         Right|
|         M. Neuer| 31|      193|       92|    Germany|  FC Bayern München|      61000000|         Right|
|        L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona|      97000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+--------------+--------------+
only showing top 5 rows



Podemos Eliminar Columnas:

In [149]:
# de forma similar a con Pandas con 'drop()'
df_fifa18.drop('weight_kg').show(5)


+-----------------+---+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
|Cristiano Ronaldo| 32|      185|   Portugal|        Real Madrid| 95500000|         Right|
|         L. Messi| 30|      170|  Argentina|       FC Barcelona|105000000|          Left|
|           Neymar| 25|      175|     Brazil|Paris Saint-Germain|123000000|         Right|
|         M. Neuer| 31|      193|    Germany|  FC Bayern München| 61000000|         Right|
|        L. Suárez| 30|      182|    Uruguay|       FC Barcelona| 97000000|         Right|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



Podemos Ordenar Columnas:

In [150]:
#Podemos ordenar a los jugadores por su valor
df_fifa18.orderBy('value_eur', ascending = False).show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|           Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|123000000|         Right|
|         L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|105000000|          Left|
|        L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona| 97000000|         Right|
|Cristiano Ronaldo| 32|      185|       80|   Portugal|        Real Madrid| 95500000|         Right|
|   R. Lewandowski| 28|      185|       79|     Poland|  FC Bayern München| 92000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



Y además podemos Filtrar Datos:

In [152]:
# Mediate la funcion where()
df_fifa18.where(df_fifa18.age > 35).show(5)
# Jugadores que tegan mas de 35 años

+-----------+---+---------+---------+-----------+--------------------+---------+--------------+
| short_name|age|height_cm|weight_kg|nationality|                club|value_eur|preferred_foot|
+-----------+---+---------+---------+-----------+--------------------+---------+--------------+
|  G. Buffon| 39|      191|       94|      Italy|            Juventus|  4500000|         Right|
|A. Barzagli| 36|      187|       87|      Italy|            Juventus|  5000000|         Right|
|     Aduriz| 36|      182|       78|      Spain|Athletic Club de ...| 11000000|         Right|
|   Casillas| 36|      185|       84|      Spain|            FC Porto|  3500000|          Left|
|   S. Eto'o| 36|      179|       75|   Cameroon|         Antalyaspor|  6500000|         Right|
+-----------+---+---------+---------+-----------+--------------------+---------+--------------+
only showing top 5 rows



In [153]:
# Y podemos concatener codiciones

# P.E. Los jugadores del Liverpool que tuviera mas de 30 años de edad
df_fifa18.where((df_fifa18.age > 30) & (df_fifa18.club == 'Liverpool')).show(5)

+----------+---+---------+---------+-----------+---------+---------+--------------+
|short_name|age|height_cm|weight_kg|nationality|     club|value_eur|preferred_foot|
+----------+---+---------+---------+-----------+---------+---------+--------------+
| J. Milner| 31|      175|       70|    England|Liverpool|  9500000|         Right|
| R. Klavan| 31|      187|       85|    Estonia|Liverpool|  7000000|          Left|
+----------+---+---------+---------+-----------+---------+---------+--------------+



*Ejemplo 1*: ¿Cuál es el costo total e euro de un equipo?

Esto se va a hacer sumando ekl valor individual de cada jugador del equipo:

In [156]:
costo_club = df_fifa18.groupBy('Club').sum('value_eur')
costo_club.show(5)

+--------------------+--------------+
|                Club|sum(value_eur)|
+--------------------+--------------+
|       Côte d'Ivoire|             0|
|             Palermo|      45975000|
|          Göztepe SK|      45225000|
|CD Everton de Viñ...|      46800000|
|            Paraguay|             0|
+--------------------+--------------+
only showing top 5 rows



In [157]:
# Y podemos ordenarlas e orden ascendete:
costo_club = costo_club.orderBy('sum(value_eur)', ascending = False)
costo_club.show(5)

+-------------------+--------------+
|               Club|sum(value_eur)|
+-------------------+--------------+
|        Real Madrid|     826750000|
|  FC Bayern München|     748825000|
|       FC Barcelona|     715000000|
|            Chelsea|     671910000|
|Paris Saint-Germain|     631750000|
+-------------------+--------------+
only showing top 5 rows



*Ejemplo 2*: ¿Cúal es el costo de cada quipo y la edad promedio de sus jugadores?

In [158]:
# Usemos el comando anterior para determinar el costo de cada equipo

club_edad = df_fifa18.groupBy('club').agg(sum('value_eur').alias('valor_club'), 
                                          mean('age').alias('edad_promedio'))

club_edad.show(5)

+--------------------+----------+------------------+
|                club|valor_club|     edad_promedio|
+--------------------+----------+------------------+
|       Côte d'Ivoire|         0|              28.2|
|             Palermo|  45975000|             24.75|
|          Göztepe SK|  45225000| 26.77777777777778|
|CD Everton de Viñ...|  46800000|              29.0|
|            Paraguay|         0|30.666666666666668|
+--------------------+----------+------------------+
only showing top 5 rows



In [159]:
# Ordenemos de forma ascendente

club_edad = club_edad.orderBy('valor_club', ascending = False)
club_edad.show(5)

+-------------------+----------+------------------+
|               club|valor_club|     edad_promedio|
+-------------------+----------+------------------+
|        Real Madrid| 826750000|24.576923076923077|
|  FC Bayern München| 748825000|25.076923076923077|
|       FC Barcelona| 715000000|             26.25|
|            Chelsea| 671910000|24.393939393939394|
|Paris Saint-Germain| 631750000|24.392857142857142|
+-------------------+----------+------------------+
only showing top 5 rows



**Ejemplo 3**: ¿Cuántos jugadores son zurdos y cuantos diestros?

In [168]:
# Vamos a ordenar a los jugadores por su pie preferido con GroupBy()
pie = df_fifa18.groupBy('preferred_foot').count()
pie.show(5)

+--------------+-----+
|preferred_foot|count|
+--------------+-----+
|          Left| 4166|
|         Right|13426|
+--------------+-----+



**Ejemplo 4**: ¿Cuál es la proporción entre diestros y zurdos en cada equipo?

In [186]:
# Vamos a generar dos DataFrame con la filtración de preferencia de pie y posteriormente vamos a 
# contar los jugadores de cada equipo

izq = df_fifa18.where(df_fifa18.preferred_foot == 'Left').groupBy('Club').agg(count('preferred_foot').alias('izq'))
der = df_fifa18.where(df_fifa18.preferred_foot == 'Right').groupBy('Club').agg(count('preferred_foot').alias('der'))

izq.show(5)
der.show(5)

+--------------------+---+
|                Club|izq|
+--------------------+---+
|             Palermo|  6|
|CD Everton de Viñ...|  7|
|          Göztepe SK|  5|
|  Santiago Wanderers|  7|
|               Carpi|  3|
+--------------------+---+
only showing top 5 rows



+--------------------+---+
|                Club|der|
+--------------------+---+
|       Côte d'Ivoire|  5|
|             Palermo| 22|
|          Göztepe SK| 22|
|CD Everton de Viñ...| 13|
|            Paraguay|  8|
+--------------------+---+
only showing top 5 rows



In [187]:
# Hagamos un join con el equipo como llave

pref_pie = izq.join(der, on = 'Club')
pref_pie.show(9)


+--------------------+---+---+
|                Club|izq|der|
+--------------------+---+---+
|             Palermo|  6| 22|
|          Göztepe SK|  5| 22|
|CD Everton de Viñ...|  7| 13|
|            Paraguay|  1|  8|
|  1. FC Union Berlin|  5| 22|
|  Santiago Wanderers|  7| 13|
|               Carpi|  3| 27|
|          Sagan Tosu|  3| 22|
|           Puebla FC| 10| 19|
+--------------------+---+---+
only showing top 9 rows



In [189]:
# Calculo de Proporcion:

pref_pie = pref_pie.withColumn('proporcion', pref_pie.izq / pref_pie.der)
pref_pie.show(5)

+--------------------+---+---+-------------------+
|                Club|izq|der|         proporcion|
+--------------------+---+---+-------------------+
|             Palermo|  6| 22| 0.2727272727272727|
|          Göztepe SK|  5| 22|0.22727272727272727|
|CD Everton de Viñ...|  7| 13| 0.5384615384615384|
|            Paraguay|  1|  8|              0.125|
|  1. FC Union Berlin|  5| 22|0.22727272727272727|
+--------------------+---+---+-------------------+
only showing top 5 rows



### Consultas en SQL

Primero importemos los datos  en el formato SQL

In [190]:
df_fifa18.registerTempTable('FIFA18')

/usr/local/lib/python3.9/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
22/02/05 19:01:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [192]:
# Y ahora podemos realizar consultas con código sql

consulta = '''
  SELECT club, SUM(value_eur) AS valor_total 
  FROM FIFA18
  GROUP BY club
  ORDER BY valor_total DESC
  '''
spark.sql(consulta).show(5)

+-------------------+-----------+
|               club|valor_total|
+-------------------+-----------+
|        Real Madrid|  826750000|
|  FC Bayern München|  748825000|
|       FC Barcelona|  715000000|
|            Chelsea|  671910000|
|Paris Saint-Germain|  631750000|
+-------------------+-----------+
only showing top 5 rows



**Ejemplo 1**: ¿Cuál es el club con menor numero dde jugadores en la base de datos?

In [194]:
# Veamos toda la tabla para ver el nombre de las columnas

consulta1 = '''
  SELECT *
  FROM FIFA18
  '''
spark.sql(consulta1).show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|Cristiano Ronaldo| 32|      185|       80|   Portugal|        Real Madrid| 95500000|         Right|
|         L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|105000000|          Left|
|           Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|123000000|         Right|
|         M. Neuer| 31|      193|       92|    Germany|  FC Bayern München| 61000000|         Right|
|        L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona| 97000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



In [196]:
# Contemos a cada jugador

consulta1 = '''
  SELECT club, COUNT(short_name) AS jugadores
  FROM FIFA18
  GROUP BY club
  ORDER BY jugadores ASC
  '''
spark.sql(consulta1).show(5)

+------------+---------+
|        club|jugadores|
+------------+---------+
|      Poland|        1|
|Halmstads BK|        1|
|    Portugal|        1|
|     Belgium|        1|
|    Colombia|        1|
+------------+---------+
only showing top 5 rows

